## Example of collecting data from CARLA

In [ ]:
import carla
import numpy as np
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)
world = client.get_world()
map = world.get_map()
spawn_points = map.get_spawn_points()
settings = world.get_settings()
ego_vehicle = None
imu_sensor = None
gnss_sensor = None
t = []
gt_p = []
gnss_data = []
imu_data = []

try:
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.005
    world.apply_settings(settings)
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(spawn_points[0].location + carla.Location(z=50), carla.Rotation(pitch=-90)))
    tesla_model_3 = world.get_blueprint_library().filter('model3')[0]
    ego_vehicle = world.spawn_actor(tesla_model_3, spawn_points[0])
    
    gnss_sensor = world.get_blueprint_library().find('sensor.other.gnss')
    gnss_sensor.set_attribute('sensor_tick', '0.1')
    gnss_sensor_transform = carla.Transform(carla.Location(x=1.0, z=2.8))
    gnss_sensor = world.spawn_actor(gnss_sensor, gnss_sensor_transform, attach_to=ego_vehicle)
    def gnss_callback(data):
        gnss_data.append([data.latitude, data.longitude, data.altitude])
    gnss_sensor.listen(lambda data: gnss_callback(data))
    
    imu_sensor = world.get_blueprint_library().find('sensor.other.imu')
    imu_sensor.set_attribute('sensor_tick', '0.1')
    imu_sensor_transform = carla.Transform(carla.Location(x=1.0, z=1.8))
    imu_sensor = world.spawn_actor(imu_sensor, imu_sensor_transform, attach_to=ego_vehicle)
    def imu_callback(data):
        imu_data.append([data.accelerometer.x, data.accelerometer.y, data.accelerometer.z, data.gyroscope.x, data.gyroscope.y, data.gyroscope.z])
    imu_sensor.listen(lambda data: imu_callback(data))
    
    world.tick()

    t.clear()
    gt_p.clear()
    gnss_data.clear()
    imu_data.clear()

    DURATION = 1
    total_time_steps = DURATION / settings.fixed_delta_seconds
    for i in range(int(total_time_steps)):
        t.append(i * settings.fixed_delta_seconds)
        loc = ego_vehicle.get_location()
        gt_p.append([loc.x, loc.y, loc.z])
        world.tick()
except Exception as e:
    print(e)
finally:
    settings.synchronous_mode = False
    world.apply_settings(settings)
    if ego_vehicle is not None:
        ego_vehicle.destroy()
    if gnss_sensor is not None:
        gnss_sensor.destroy()
    if imu_sensor is not None:
        imu_sensor.destroy()

In [ ]:
from data.data import Data
from data.utils import StampedData
created_gt_data = Data(t=np.array(t), p=np.array(gt_p))

imu_data_stamped = StampedData()
imu_data_stamped.t = t
imu_data_stamped.data = imu_data

gnss_data_stamped = StampedData()
gnss_data_stamped.t = t
gnss_data_stamped.data = gnss_data